In [22]:
import os
from langchain.llms.openai import OpenAI
from langchain.sql_database import SQLDatabase
from dotenv import load_dotenv # pip install load-dotenv
load_dotenv()

import psycopg2 # pip install psycopg-binary

In [23]:
username="postgres"
password = "postgres"
host="localhost"
database= "postgres"
CONNECTION_STRING = f"postgresql+psycopg2://{username}:{password}@{host}/{database}"

db = SQLDatabase.from_uri(CONNECTION_STRING)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM public.knowledge")

postgresql
['knowledge']


"[('knowledge name 1', 'knowledge body 1'), ('knowledge name 2', 'knowledge body 2'), ('knowledge name 3', 'knowledge body 3'), ('knowledge name 4', 'knowledge body 4'), ('knowledge name 5', 'knowledge body 5'), ('knowledge name 6', 'knowledge body 6'), ('knowledge name 7', 'knowledge body 7'), ('knowledge name 8', 'knowledge body 8'), ('knowledge name 9', 'knowledge body 9'), ('knowledge name 10', 'knowledge body 10')]"

In [24]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many knowledge are there"})
response

'SELECT COUNT(*) AS total_knowledge FROM knowledge;'

In [25]:
db.run(response)



'[(10,)]'